In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [17]:
from langchain.agents import create_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model = "qwen/qwen3-next-80b-a3b-instruct", temperature=0.2)

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [18]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt=("You are a helpful assistant who uses subagents to calculate the square root or square of a number. "
        "After you receive the result from a subagent, you MUST use that result to provide a final, clear answer to the user's original question.")
)

## Test

In [19]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [20]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='49dfd56a-19cf-4fa7-ada9-0955ecd8db1a'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b683972f9e744692a3c5bdde', 'index': -1, 'type': 'function', 'function': {'name': 'call_subagent_1', 'arguments': '{"x": 456}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'reasoning_content': None, 'tool_calls': [{'id': 'call_b683972f9e744692a3c5bdde', 'index': -1, 'type': 'function', 'function': {'name': 'call_subagent_1', 'arguments': '{"x": 456}'}}], 'token_usage': {'prompt_tokens': 275, 'total_tokens': 300, 'completion_tokens': 25, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'finish_reason': 'tool_calls', 'model_name': 'qwen/qwen3-next-80b-a3b-instruct'}, id='lc_run--019c8d9a-dda4-7430-8b68-019a09a310bc-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_b683972f9e744692a3c5bdde', 'type': 'tool